In [ ]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered, save_output
from marker.config.parser import ConfigParser
import pymupdf4llm
from statistics import mean
import time
import os
import pathlib
from docling.document_converter import DocumentConverter

# Tekstextractie uit PDF-bestanden - Markdown-formaat

## 1. Doelstellingen

De meeste informatiebronnen waarover we beschikken zijn in PDF-formaat, of kunnen eenvoudig naar PDF worden omgezet. Aangezien een RAG-pijplijn werkt met tekst en niet met bestanden, is het belangrijk dat we de inhoud van deze documenten kunnen extraheren.

Een eenvoudige extractie van platte tekst is echter geen ideale aanpak. PDF-bestanden bevatten vaak een duidelijke structuur – zoals titels, subtitels, lijsten en tabellen – die semantische informatie verbergt. Bij blinde extractie gaat die structuur verloren, waardoor het moeilijker wordt om samenhangende en betekenisvolle chunks aan te maken. Dit vergroot de kans dat de LLM in verwarring wordt gebracht in relatie tot de gestelde vraag.

Daarom richten we ons tot Markdown. Markdown is een eenvoudige opmaaktaal waarmee de structuur van een document expliciet wordt weergegeven in de tekst zelf. Door PDF-bestanden om te zetten naar Markdown kunnen we de oorspronkelijke opbouw van een document grotendeels behouden. Dit helpt de LLM om de informatie beter te begrijpen en relevantere antwoorden te genereren.

In dit notebook onderzoeken we drie tools – [Marker](https://github.com/VikParuchuri/marker), [PyMuPDF4LLM](https://pymupdf.readthedocs.io/en/latest/pymupdf4llm/) en [Docling](https://docling-project.github.io/docling/) – die allemaal beweren in staat te zijn om dit te doen. We willen daarbij het volgende nagaan:

- Hoe worden de respectieve tools gebruikt? Zijn ze gebruiksvriendelijk?
- Hoeveel tijd neemt de omzetting in beslag?
- Is de gegenereerde Markdown van goede kwaliteit?

## 2. Methodologie

Om de bruikbaarheid en efficiëntie — met andere woorden, de kwaliteit van de gegenereerde Markdown en de tijd die de omzetting in beslag neemt — van de tools te evalueren, zullen we vier types PDF-bestanden omzetten. Bij elke omzetting meten we de benodigde tijd met behulp van de `time`-module, en beoordelen we in welke mate de gegenereerde Markdown de structuur van het oorspronkelijke document weet te behouden.

De vier bestandssoorten die we testen zijn:

- **Studiewijzers**: Deze zijn beschikbaar via Chamilo en maken dus deel uit van een webpagina. Chamilo biedt de mogelijkheid om studiewijzers te exporteren naar PDF-formaat.
- **Slides**: Dit zijn PDF-bestanden die rechtstreeks door docenten worden aangeleverd en de leerstof van een vak bevatten. Er is hierbij geen verdere verwerking nodig.
- **Bijzondere slides**: Voor het vak *Infrastructure Automation* (en ook voor AI & Data Science) zijn de slides oorspronkelijk geschreven in Markdown, omgezet via Pandoc naar presentaties en gepubliceerd via GitHub Pages. De originele Markdown-bestanden zijn beschikbaar op GitHub. In dit geval willen we nagaan of de tools een resultaat kunnen genereren dat vergelijkbaar is met de oorspronkelijke Markdown.
- **datalinux.pdf**: Dit is de syllabus van het vak Linux for Data Scientists. Het is een omvangrijk document van ongeveer 300 pagina’s. Het doel hier is om de tools tot het uiterste te testen en te observeren hoeveel tijd ze nodig hebben om een dergelijk groot bestand te verwerken.

## 3. Uittesten

In [3]:
pdfs_path = './pdfs/'
markdowns_path = './markdowns/'

studiewijzers_pdf_path = os.path.join(pdfs_path, 'Studiewijzers/')
slides_pdf_path = os.path.join(pdfs_path, 'Slides/')
special_slides_pdf_path = os.path.join(pdfs_path, 'Bijzondere_slides/')
syllabus_pdf_path = os.path.join(pdfs_path, 'Syllabus/')

In [4]:
def chrono(path_to_pdfs, path_to_markdowns, save_markdown):
    time_taken = {}
    times = []
    
    for e in os.scandir(path_to_pdfs):
        start = time.time()
        save_markdown(e, path_to_markdowns)
        end = time.time()
        
        time_taken[e.name[:-4]] = end-start
    
    for f, t in time_taken.items():
        print(f'{f}: {t}s')
        times.append(t)

    print(f'\nTotal time taken: {sum(times)} s')
    print(f'Average time taken: {mean(times)} s\n')

### 3.1. Marker

In [5]:
marker_markdowns_path = os.path.join(markdowns_path, 'Marker/')

In [6]:
# Aanmaken van de configuratie-object. Deze object wordt intern 
# door Marker gebruikt om na te gaan hoe de omzetting moet gebeuren.
# De gehele lijst van opties kan bekeken worden door marker_single --help 
# in een console te tijpen
config = { # Dictionary met de opties en hun waardenn
    'output_format': 'markdown',
    'disable_image_extraction': True,
}

config_parser = ConfigParser(config) # Aanmaken van de configuratie-object

In [7]:
# Aanmaken van de omzettingsobject. 

# Nota: soms kan hier een probleem ontstaan dat aangeeft dat een bepaalde
# bestand al reeds bestaat. Dit is een teken dat conda geen Administrator
# rechten heeft en kan opgelost worden door alles als Administator te runnen

converter = PdfConverter(
    artifact_dict=create_model_dict(),
    config=config_parser.generate_config_dict(),
)

Loaded layout model s3://layout/2025_02_18 on device cpu with dtype torch.float32
Loaded texify model s3://texify/2025_02_18 on device cpu with dtype torch.float32
Loaded recognition model s3://text_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded table recognition model s3://table_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded detection model s3://text_detection/2025_02_28 on device cpu with dtype torch.float32
Loaded detection model s3://inline_math_detection/2025_02_24 on device cpu with dtype torch.float32


In [8]:
def marker_save_markdown(e, path_to_markdowns):
    rendered = converter(e.path)
    save_output(rendered, path_to_markdowns, e.name[:-4])

#### 3.1.1. Studiewijzers

In [9]:
studiewijzers_markdowns_path = os.path.join(marker_markdowns_path, 'Studiewijzers/')

In [10]:
chrono(studiewijzers_pdf_path, studiewijzers_markdowns_path, marker_save_markdown)

Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 8/8 [00:08<00:00,  1.01s/it]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.03s/it]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|█

Studiewijzer_AI_Data_Science: 147.66392993927002s
Studiewijzer_Infrastructure_Automation: 44.52555823326111s
Studiewijzer_linux_for_Data_Scientists: 45.2026903629303s

Total time taken: 237.39217853546143 s
Average time taken: 79.13072617848714 s



#### 3.1.2. Slides

In [12]:
slides_markdowns_path = os.path.join(marker_markdowns_path, 'Slides/')

In [13]:
chrono(slides_pdf_path, slides_markdowns_path, marker_save_markdown)

Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.49it/s]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 8/8 [00:01<00:00,  4.31it/s]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Detecting bboxes: 0it [00:00, ?it/s

dsai-en-0-intro: 59.582258224487305s
dsai-en-1-sampling: 76.04399728775024s
dsai-en-2-univariate: 99.138028383255s
dsai-en-3a-central-limit-theorem: 318.20600867271423s
dsai-en-3b-hypothesis-testing: 124.24505090713501s
dsai-en-4-bivariate-qual-qual: 152.80231857299805s
dsai-en-5-bivariate-qual-quant: 108.76130723953247s
dsai-en-6-bivariate-quant-quant: 204.01630187034607s
dsai-en-7-timeseries: 141.74169158935547s

Total time taken: 1284.5369627475739 s
Average time taken: 142.72632919417487 s



#### 3.1.3. Bijzondere slides

In [16]:
special_slides_markdowns_path = os.path.join(marker_markdowns_path, 'Bijzondere_slides/')

In [17]:
chrono(special_slides_pdf_path, special_slides_markdowns_path, marker_save_markdown)

Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.56it/s]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Running OCR Error Detection: 100%|███████████████████████████████████████████████████████| 8/8 [00:05<00:00,  1.49it/s]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
Detecting bboxes: 0it [00:00, ?it/s]
Running OCR Error Detection: 100%|█

1. Continuous Integration_ Deployment with Jenkins: 132.40918135643005s
2. Configuration Management with Ansible: 75.54081463813782s
3. Container orchestration with Kubernetes: 75.29476118087769s
4. Monitoring with Prometheus: 397.08021068573s
Infrastructure Automation_ inleiding: 89.35674405097961s

Total time taken: 769.6817119121552 s
Average time taken: 153.93634238243104 s



#### 3.1.4. Syllabus

In [18]:
syllabus_markdowns_path = os.path.join(marker_markdowns_path, 'Syllabus/')

In [19]:
chrono(syllabus_pdf_path, syllabus_markdowns_path, marker_save_markdown)

Running OCR Error Detection: 100%|█████████████████████████████████████████████████████| 75/75 [01:22<00:00,  1.10s/it]
Detecting bboxes: 0it [00:00, ?it/s]
C:\ProgramData\anaconda3\envs\Bachelorproef\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
Recognizing tables: 100%|██████████████████████████████████████████████████████████████| 14/14 [03:10<00:00, 13.61s/it]


datalinux: 1457.6885430812836s

Total time taken: 1457.6885430812836 s
Average time taken: 1457.6885430812836 s



### 3.2. PyMuPDF4LLM

In [20]:
pymupdf4llm_markdowns_path = os.path.join(markdowns_path, 'PyMuPDF4LLM/')

In [21]:
def pymupdf4llm_save_markdown(e, path_to_markdowns):
    md =  pymupdf4llm.to_markdown(e.path, write_images = False, show_progress = True)
    filename = f'{e.name[:-4]}.md'
    pathlib.Path(os.path.join(path_to_markdowns, filename)).write_bytes(md.encode())

#### 3.2.1. Studiewijzers

In [22]:
studiewijzers_markdowns_path = os.path.join(pymupdf4llm_markdowns_path, 'Studiewijzers/')

In [23]:
chrono(studiewijzers_pdf_path, studiewijzers_markdowns_path, pymupdf4llm_save_markdown)

Processing ./pdfs/Studiewijzers/Studiewijzer_AI_Data_Science.pdf...
]========================================] (30/30)
Processing ./pdfs/Studiewijzers/Studiewijzer_Infrastructure_Automation.pdf...
]========================================] (15/15)
Processing ./pdfs/Studiewijzers/Studiewijzer_linux_for_Data_Scientists.pdf...
]========================================] (14/14)
Studiewijzer_AI_Data_Science: 7.71200704574585s
Studiewijzer_Infrastructure_Automation: 3.1662962436676025s
Studiewijzer_linux_for_Data_Scientists: 4.027928352355957s

Total time taken: 14.90623164176941 s
Average time taken: 4.968743880589803 s



#### 3.2.2. Slides

In [24]:
slides_markdowns_path = os.path.join(pymupdf4llm_markdowns_path, 'Slides/')

In [25]:
chrono(slides_pdf_path, slides_markdowns_path , pymupdf4llm_save_markdown)

Processing ./pdfs/Slides/dsai-en-0-intro.pdf...
]========================================] (27/27)
Processing ./pdfs/Slides/dsai-en-1-sampling.pdf...
]========================================] (30/30)
Processing ./pdfs/Slides/dsai-en-2-univariate.pdf...
]========================================] (37/37)
Processing ./pdfs/Slides/dsai-en-3a-central-limit-theorem.pdf...
]========================================] (75/75)
Processing ./pdfs/Slides/dsai-en-3b-hypothesis-testing.pdf...
]========================================] (38/38)
Processing ./pdfs/Slides/dsai-en-4-bivariate-qual-qual.pdf...
]========================================] (46/46)
Processing ./pdfs/Slides/dsai-en-5-bivariate-qual-quant.pdf...
]========================================] (31/31)
Processing ./pdfs/Slides/dsai-en-6-bivariate-quant-quant.pdf...
]========================================] (28/28)
Processing ./pdfs/Slides/dsai-en-7-timeseries.pdf...
]========================================] (39/39)
dsai-en-0-intro: 0.5

#### 3.2.3. Bijzondere slides

In [26]:
special_slides_markdowns_path = os.path.join(pymupdf4llm_markdowns_path, 'Bijzondere_slides/')

In [27]:
chrono(special_slides_pdf_path, special_slides_markdowns_path , pymupdf4llm_save_markdown)

Processing ./pdfs/Bijzondere_slides/1. Continuous Integration_ Deployment with Jenkins.pdf...
]========================================] (26/26)
Processing ./pdfs/Bijzondere_slides/2. Configuration Management with Ansible.pdf...
]========================================] (32/32)
Processing ./pdfs/Bijzondere_slides/3. Container orchestration with Kubernetes.pdf...
]========================================] (33/33)
Processing ./pdfs/Bijzondere_slides/4. Monitoring with Prometheus.pdf...
]========================================] (29/29)
Processing ./pdfs/Bijzondere_slides/Infrastructure Automation_ inleiding.pdf...
]========================================] (40/40)
1. Continuous Integration_ Deployment with Jenkins: 0.4896726608276367s
2. Configuration Management with Ansible: 0.5166478157043457s
3. Container orchestration with Kubernetes: 0.716853141784668s
4. Monitoring with Prometheus: 0.4909341335296631s
Infrastructure Automation_ inleiding: 0.810161828994751s

Total time taken: 3.02

#### 3.2.4. Syllabus

In [28]:
syllabus_markdowns_path = os.path.join(pymupdf4llm_markdowns_path, 'Syllabus/')

In [29]:
chrono(syllabus_pdf_path, syllabus_markdowns_path , pymupdf4llm_save_markdown)

Processing ./pdfs/Syllabus/datalinux.pdf...
]========================================] (300/300)
datalinux: 16.896792888641357s

Total time taken: 16.896792888641357 s
Average time taken: 16.896792888641357 s



### 3.3. Docling

In [34]:
docling_markdowns_path = os.path.join(markdowns_path, 'Docling/')

In [35]:
def docling_save_markdown(e, path_to_markdowns):
    converter = DocumentConverter()
    md =  converter.convert(e.path)
    md.document.save_as_markdown(f'{os.path.join(path_to_markdowns, e.name[:-4])}.md', image_placeholder='')

#### 3.3.1. Studiewijzers

In [36]:
studiewijzers_markdowns_path = os.path.join(docling_markdowns_path, 'Studiewijzers/')

In [37]:
chrono(studiewijzers_pdf_path, studiewijzers_markdowns_path, docling_save_markdown)

Studiewijzer_AI_Data_Science: 95.69085454940796s
Studiewijzer_Infrastructure_Automation: 49.91907000541687s
Studiewijzer_linux_for_Data_Scientists: 43.21861004829407s

Total time taken: 188.8285346031189 s
Average time taken: 62.9428448677063 s



#### 3.3.2. Slides

In [38]:
slides_markdowns_path = os.path.join(docling_markdowns_path, 'Slides/')

In [39]:
chrono(slides_pdf_path, slides_markdowns_path, docling_save_markdown)

dsai-en-0-intro: 50.02930521965027s
dsai-en-1-sampling: 66.7761583328247s
dsai-en-2-univariate: 73.77223348617554s
dsai-en-3a-central-limit-theorem: 135.45645141601562s
dsai-en-3b-hypothesis-testing: 67.05923223495483s
dsai-en-4-bivariate-qual-qual: 102.2901918888092s
dsai-en-5-bivariate-qual-quant: 64.01545023918152s
dsai-en-6-bivariate-quant-quant: 57.748496294021606s
dsai-en-7-timeseries: 68.35293292999268s

Total time taken: 685.500452041626 s
Average time taken: 76.16671689351399 s



#### 3.3.3. Bijzondere slides

In [40]:
special_slides_markdowns_path = os.path.join(docling_markdowns_path, 'Bijzondere_slides/')

In [41]:
chrono(special_slides_pdf_path, special_slides_markdowns_path, docling_save_markdown)

1. Continuous Integration_ Deployment with Jenkins: 70.95459246635437s
2. Configuration Management with Ansible: 71.51973104476929s
3. Container orchestration with Kubernetes: 64.15011310577393s
4. Monitoring with Prometheus: 127.50278973579407s
Infrastructure Automation_ inleiding: 112.22828364372253s

Total time taken: 446.3555099964142 s
Average time taken: 89.27110199928283 s



#### 3.3.4. Syllabus

In [42]:
syllabus_markdowns_path = os.path.join(docling_markdowns_path, 'Syllabus/')

In [43]:
chrono(syllabus_pdf_path, syllabus_markdowns_path, docling_save_markdown)

datalinux: 837.9995594024658s

Total time taken: 837.9995594024658 s
Average time taken: 837.9995594024658 s



## 4. Besluit

> **Opgelet**: De cijfers die hier besproken worden, zijn geldig op 20 april 2025. We kunnen hun nauwkeurigheid na deze datum niet garanderen.

Wat meteen opvalt, is de snelheid waarmee PyMuPDF4LLM de omzettingen uitvoert. De verwerkingstijden zijn indrukwekkend:

- Gemiddeld 5 seconden per studiewijzer
- Gemiddeld 0,8 seconden per slides
- Gemiddeld 0,6 seconden per bijzondere slides
- Gemiddeld 17 seconden voor de syllabus van 

Geen van de andere tools komt zelfs in de buurt van deze snelheid. Helaas, is de kwaliteit van de gegenereerde Markdown niet overtuigend. PyMuPDF4LLM slaagt er niet in om de structuur van het oorspronkelijke PDF-bestand betrouwbaar te reconstrueren. Bovendien, neemt de tool tekstregels letterlijk over zoals ze in het PDF-bestand voorkomen, zonder rekening te houden met zinnen die door een pagina- of regelovergang onderbroken zijn. Daardoor wordt de natuurlijke opbouw van de tekst soms verstoord, wat een negatieve impact kan hebben op de leesbaarheid en interpretatie door een LLM.

Wat PyMuPDF4LLM wel goed doet is het begrijpen van wiskundige symbolen. Voor de slides van het vak AI & Data Science, hebben we geen verkeerde mathematische symbolen opgemerkt in de gegenereerde Markdown. Alles lijkt correct overgenomen te zijn. Ondanks dit, verkiezen we toch niet voor PyMuPDF4LLM omdat structuur veel zwaarder weegt dan mathematische symbolen voor dit toepassing.

Marker en Docling zijn beide geavanceerdere tools dan PyMuPDF4LLM en maken bij de omzetting gebruik van meer verfijnde technieken.
Als we kijken naar de gemiddelde omzettingstijden, blijkt dat Docling sneller is dan Marker.
Voor de omzetting van de syllabus van Linux for Data Scientists is Docling maar liefst 620 seconden sneller dan Marker — dat is meer dan 10 minuten verschil, wat een aanzienlijk tijdsvoordeel oplevert.

|                  | Marker | Docling |
|------------------|--------|---------|
| Studiewijzers    | 79s    | 63s     |
| Slides           | 143s   | 76s     |
| Bijzonder slides | 153s   | 89s     |
| Syllabus         | 1457s  | 837s    |

Het probleem van Docling ligt bij de gegenereerde Markdown. Docling is niet altijd in staat om de gehele inhoud te extraheren. Dit deed zich voor bij de slides van het vak AI & Data Science over de centrale limietstelling (zie resulterende Markdown [hier](./markdowns/Docling/Slides/dsai-en-3a-central-limit-theorem.md))

Hoevel Marker trager is, hebben we dergelijke probleem niet ondervonden. Bovendien, slaagt Marker om de structuur van de originele PDF's maximaal te behouden. Daarom verkiezen we Marker boven de andere tools.